In [1]:
import os
import sys
import numpy as np
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
from eloguessr import EloGuessr
from utils import load_data, plot_losses

torch.manual_seed(42)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [2]:
data_dir = '/Home/siv33/vbo084/EloGuessr/data/processed'
model_dir = '/Home/siv33/vbo084/EloGuessr/models/final_models'

In [3]:
model = torch.load(os.path.join(model_dir, 'model_elite_medium.pt'))

#fnames = ['chess_train_elite_full.pt', 'chess_val_elite_full.pt', 'chess_test_elite_full.pt']
fnames = ['chess_train_elite_medium.pt', 'chess_val_elite_medium.pt', 'chess_test_elite_medium.pt']
#fnames = ['chess_train_both_full.pt', 'chess_val_both_full.pt', 'chess_test_both_full.pt']

BATCH_SIZE = 64
train_dloader, val_dloader, test_dloader = load_data(data_dir, fnames, batch_size=BATCH_SIZE)
del train_dloader

In [10]:
def eval(model_list: list, dataloader):
    pred_scores = []
    true_scores = []

    model.eval()
    with torch.no_grad():
        for batch in dataloader:
            inputs, targets = batch
            inputs = inputs.to(device)
            targets = targets.to(device)

            scores = model(inputs)
            pred_scores.append(scores.squeeze())  # Ensure scores are in the right shape
            true_scores.append(targets.squeeze())  # Ensure targets are in the right shape


    pred_scores = torch.cat(pred_scores, dim=0)
    true_scores = torch.cat(true_scores, dim=0)
    print(f'Pred scores before denorm: {pred_scores[:5]}')

    assert pred_scores.shape == true_scores.shape, f"Shape mismatch: {pred_scores.shape} vs {true_scores.shape}"

    pred_scores = pred_scores * (3000 - 100) + 100 # Denormalize the scores
    true_scores = true_scores * (3000 - 100) + 100

    print(f'Pred scores AFTER denorm: {pred_scores[:5]}')

    print(pred_scores)
    print(true_scores)

    hits = [25, 50, 100, 250]
    hit_percentages = {}
    for hit in hits:
        hits_x = torch.abs(pred_scores - true_scores) <= hit
        hit_percentage = hits_x.sum().item() / len(hits_x) * 100
        hit_percentages[f'Hits@{hit}'] = hit_percentage

    for key, value in hit_percentages.items():
        print(f'{key}: {value:.2f}%')

    return hit_percentages

In [11]:
models = [model]

In [12]:
evals = eval(models, val_dloader)

Pred scores before denorm: tensor([0.8686, 0.8650, 0.8814, 0.8687, 0.8902], device='cuda:0')
Pred scores AFTER denorm: tensor([2618.8950, 2608.4871, 2656.1438, 2619.1987, 2681.6880],
       device='cuda:0')
tensor([2618.8950, 2608.4871, 2656.1438,  ..., 2639.6667, 2636.8279,
        2613.8413], device='cuda:0')
tensor([2436.0000, 2353.0000, 2476.5000,  ..., 2359.5000, 2410.0000,
        2340.0000], device='cuda:0')
Hits@25: 2.30%
Hits@50: 4.67%
Hits@100: 12.39%
Hits@250: 73.26%
